# Introduction

Attached is a genes X count matrix for micro RNAs from HepG2.  

I am trying to select a Browser shot example that has an expressed microRNA near a collection of other regulatory 
features that I have chosen from STARR-seq and TF-ChIP-seq data.  

It would really help if I had the chromosome coordinates (in Browser address form) for all of these
microRNAs.  Can you map chromosome coordinates from hg38 onto these ENSEMBLE IDs?


In [1]:
import pandas
import numpy
from pathlib import Path
import os
import sys

In [2]:
WRS = str(Path("~/proj/woldlab-rna-seq").expanduser())
if WRS not in sys.path:
    sys.path.append(WRS)
    
from woldrnaseq.compute_read_distribution import read_tabix


In [3]:
spreadsheet = "HepG2_Ljungman_growth_microRNA_quantifications.xlsx"

In [4]:
genome_dir = Path("~/proj/genome").expanduser()
gtf_name = genome_dir / "GRCh38-V29-male" / "GRCh38-V29-male.h5"

In [5]:
store = pandas.HDFStore(gtf_name)
gtf_key = list(store.keys())[0]
gtf = store.select(
    gtf_key,
    where='type="{}"'.format("gene"),
    columns=["chromosome", "type", "start", "stop", "strand", "gene_id"],
)
store.close()
gtf["coordinate"] = gtf.apply(lambda row: "{}:{}-{}".format(row["chromosome"], row["start"], row["stop"]), axis=1)
gtf.shape

(58780, 7)

In [6]:
gtf.head()

,chromosome,type,start,stop,strand,gene_id,coordinate
0,chr1,gene,11869,14409,1,ENSG00000223972.5,chr1:11869-14409
12,chr1,gene,14404,29570,-1,ENSG00000227232.5,chr1:14404-29570
25,chr1,gene,17369,17436,-1,ENSG00000278267.1,chr1:17369-17436
28,chr1,gene,29554,31109,1,ENSG00000243485.5,chr1:29554-31109
36,chr1,gene,30366,30503,1,ENSG00000284332.1,chr1:30366-30503


In [7]:
counts = pandas.read_excel(spreadsheet, skiprows=4, names=["gene_id", "U", "+", "-"])
counts.shape

(1880, 4)

In [8]:
counts.head()

,gene_id,U,+,-
0,ENSG00000284332.1,0,0,0
1,ENSG00000273874.1,0,0,0
2,ENSG00000278791.1,0,0,0
3,ENSG00000207730.3,29934,29934,0
4,ENSG00000207607.3,4996,4876,120


In [9]:
print("gene_ids in common", len(set(counts['gene_id']).intersection(gtf["gene_id"])))
print("gene_ids different", len(set(counts['gene_id']).symmetric_difference(gtf["gene_id"])))

gene_ids in common 1880
gene_ids different 56900


In [11]:
merged = counts.merge(gtf[["gene_id", "coordinate"]], left_on="gene_id", right_on="gene_id")

In [12]:
merged.to_excel("HepG2_Ljungman_growth_microRNA_quantifications_coordinates.xlsx", index=False)